# Azure AI Foundry の基礎

このノートでは、Azure AI Foundry の Azure AI Project SDK を使ってアプリケーションを開発する上で必要な項目について学習します。


## 前提条件

このノートの内容を学習するにあたって、以下が必要になります。

- Visial Studio Code または このノートブックを実行可能な Jupyter が動作する環境
- Azure サブスクリプション
- 作成済みの Azure AI Foundry プロジェクトとデプロイしたモデル
- Python 3.8 以降
- Python に関する基礎知識
- 必要な Python パッケージ（下記のコードでインストールする）


In [ ]:
# 必要な Python パッケージのインストール
%pip install python-dotenv azure-identity azure-ai-projects

## Azure AI Project SDK

Azure AI Project SDK では以下の機能のための API が提供されています。

- プロジェクトに接続されたリソースの管理
  - Azure AI Foundry プロジェクト内の接続を列挙し、接続プロパティを取得します。
  - 例えば、Azure OpenAI接続に関連する推論エンドポイントURLや認証情報を取得します。
- 認証された推論クライアントの取得
  - 認証された推論クライアントを取得して、Azure AI Foundry プロジェクト内のデフォルトの Azure OpenAI または AIサービスへの接続を使ってチャット補完を行います。
  - `openai` パッケージの Azure Open AI クライアント、または、`azure-ai-inference` パッケージのクライアントをサポートします。
- AI エージェントの開発
  - OpenAI, Microsoft, その他の LLM プロバイダーから提供されているモデル、ツール、および、機能が広範囲に提供されたエコシステムを活用して、Azure AI Agent Service を使用してエージェントを開発します。
  - Azure AI Agent Service は、幅広い生成 AI ユースケースのためのエージェントの構築を可能にします。
  - このパッケージは現在プレビュー中です。
- AI アプリケーションの評価の実行
  - 様々な評価ロジックと指標を使用して、生成 AI アプリケーションのパフォーマンスを評価するための評価を実行します。
  - 品質、リスク、安全性のための組み込み評価ロジックが含まれており、特定のニーズに合わせたカスタム評価ロジックを含めることもできます。
- AI アプリケーションの監視
  - OpenTelemetry トレースを有効にすることができます。


### 1. Azure AI Foundry プロジェクトへの接続のための資格情報の設定

Azure AI Foundry プロジェクトへの接続のための認証を行うには、Microsoft Entra ID のユーザーが必要となります。
Azure AI Foundry プロジェクトを使ったアプリケーションでは、`TokenCredential` インターフェイスを実装するオブジェクトが必要となります。
このノートのコード サンプルでは、​​`DefaultAzureCredential` を使用しています。これを実行するには、以下が必要となります：

- 共同作成者 (Contributor) ロール
  - 接続するユーザーにこのロールの割り当ては、Azure ポータルの Azure AI プロジェクト リソースの [アクセス制御 (IAM)] タブから行うことができます。
- Azure CLI がインストールされていること
- az login を実行して、Azure アカウントにログインしていること
- 複数の Azure サブスクリプションがある場合は、Azure AI プロジェクト リソースを含むサブスクリプションが既定のサブスクリプションである必要があります。
  以下のコマンドを実行して、すべてのサブスクリプションの一覧を表示し、どのサブスクリプションが既定の設定であるかを確認します。

  ```bash
  az account list --output table
  ```

  以下のコマンドを実行して、既定のサブスクリプションを変更することができます。

  ```bash
  az account set --subscription "Your Subscription ID or Name"
  ```


In [ ]:
from azure.identity import DefaultAzureCredential

# Azure 資格情報の初期化
try:
    credential = DefaultAzureCredential()
    print(f"✓ Azure 資格情報の初期化が成功しました.")
except Exception as e:
    print(f"✗ Azure 資格情報の初期化が以下の理由で失敗しました: {str(e)}")

### 2. Azure AI Foundry プロジェクトへの認証と接続

`AIProjectClient` のクラスファクトリメソッド `from_connection_string` を使って AI Project クライアントを作成します。
`from_connection_string` では、環境変数 `PROJECT_CONNECTION_STRING` で指定したプロジェクト接続名を使用します。

`AIProjectClient` の同期クライアントを作成するには、次の手順を実行します：


In [ ]:
import os
from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient

# 環境変数の読み込み
load_dotenv('.env', override=True)

try:
    project_client = AIProjectClient.from_connection_string(
        conn_str=os.getenv("PROJECT_CONNECTION_STRING"),
        credential=credential,
    )
    print(f"✓ Azure AI プロジェクト クライアント (AIProjectClient) の初期化が成功しました.")
except Exception as e:
    print(
        f"✗ Azure AI プロジェクト クライアント (AIProjectClient) の初期化が以下の理由で失敗しました: {str(e)}"
    )

非同期クライアントを作成する場合は、追加パッケージ `aiohttp` をインストールします。


In [ ]:
%pip install aiohttp

非同期クライアントを作成する場合は、上記のコードサンプルも更新します。
`asyncio` をインポートして、`azure.ai.projects.aio` 名前空間から `AIProjectClient` をインポートします。


In [ ]:
import os
import asyncio
from dotenv import load_dotenv
from azure.ai.projects.aio import AIProjectClient

# 環境変数の読み込み
load_dotenv('.env', override=True)

try:
    project_client_async = AIProjectClient.from_connection_string(
        conn_str=os.getenv("PROJECT_CONNECTION_STRING"),
        credential=credential,
    )
    print(f"✓ Azure AI プロジェクト クライアント (AIProjectClient) の初期化が成功しました.")
except Exception as e:
    print(
        f"✗ Azure AI プロジェクト クライアント (AIProjectClient) の初期化が以下の理由で失敗しました: {str(e)}"
    )

### 3. Azure AI Foundry プロジェクトの接続リソースの確認

Azure AI Foundry ポータルでは [管理センター] が用意されています。
[管理センター] を開くと、プロジェクトに [接続されたリソース (Connected Resources)] というタブが表示されます。
AI プロジェクト クライアントの `connections` を使用すると、接続されたリソースを列挙し、それぞれの接続プロパティ (`ConnectionProperties`) を取得できます。
接続プロパティ (`ConnectionProperties`) には、リソースのエンドポイント URL や認証資格情報などが含まれます。


#### 3.1 すべての接続のプロパティを取得する

Azure AI Foundry プロジェクト内のすべての接続のプロパティを一覧表示するには以下を実行します。


In [ ]:
# List the properties of all connections of AIProject
connections = project_client.connections.list()
print(f"==> AI プロジェクトのすべての接続を列挙します ({len(connections)} 個の接続が見つかりました):")
for connection in connections:
    print(connection)

for connection in connections:
    print(f" - {connection.name}")
    print(f"    connection_type:     {connection.connection_type}")
    print(f"    authentication_type: {connection.authentication_type}")
    print(f"    endpoint_url:        {connection.endpoint_url}")

#### 3.2 特定のタイプのすべての接続のプロパティを取得する

Azure AI Foundry プロジェクト内の特定のタイプの接続のプロパティを一覧表示するには以下を実行します。
ここでは Azure OpenAI サービスのタイプを特定する。


In [ ]:
from azure.ai.projects.models import ConnectionType

connections = project_client.connections.list(
    connection_type=ConnectionType.AZURE_OPEN_AI,
)
for connection in connections:
    print(connection)

#### 3.3 デフォルト接続のプロパティを取得する

特定のタイプ (ここでは Azure OpenAI) のデフォルト接続のプロパティと視覚情報を取得する。

`include_credentials=True` で呼び出しが行われた場合、`connection.authentication_type` の値に応じて、`connection.key` または `connection.token_credential` のいずれかが挿入されます。それ以外の場合は、両方とも `None` (`null`) になります。


In [ ]:
connection = project_client.connections.get_default(
    connection_type=ConnectionType.AZURE_OPEN_AI,
    include_credentials=True,  # Optional. Defaults to "False".
)
print(connection)

#### 3.4 接続名で接続のプロパティを取得する

`connection_name` という名前の接続の接続プロパティを取得する。


In [ ]:
connection_name = "AzureAISearch"
connection = project_client.connections.get(
    connection_name=connection_name,
    include_credentials=True  # Optional. Defaults to "False"
)
print(connection)

### 4. 認証済みの ChatCompletionsClient を取得する

Azure AI Foundry プロジェクトには、チャット補完をサポートする 1 つ以上の AI モデルがデプロイされている場合があります。

これらは、OpenAI のモデル、Microsoft のモデル、または、他のプロバイダーのモデルです。
以下のコードサンプルを使用して、`azure-ai-inference` パッケージの `get_chat_completions_client` を使って、既に認証済みの `ChatCompletionsClient` を取得し、チャット補完呼び出しを実行します。

`azure-ai-inference` パッケージをインストールして、コードサンプルを実行します。

`get_chat_completions_client` では、`MODEL_NAME` は `.env` ファイルに記載したモデルのデプロイ名で実行します。


In [ ]:
%pip install azure-ai-inference

In [ ]:
from azure.ai.inference.models import UserMessage

inference_client = project_client.inference.get_chat_completions_client()

response = inference_client.complete(
    model=os.getenv("MODEL_NAME"),  # Model deployment name
    messages=[UserMessage(content="1マイルは何フィートですか?")],
)
print(response.choices[0].message.content)

### 5. 認証済みの AzureOpenAI クライアントを取得する

Azure AI Foundry プロジェクトには、チャット補完をサポートする 1 つ以上の OpenAI モデルがデプロイされている場合があります。

以下のコードを使用して、`openai` パッケージから既に認証済みの AzureOpenAI を取得し、チャット補完呼び出しを実行します。

`openai` パッケージをインストールして、コードサンプルを実行します。

`get_chat_completions_client` では、`MODEL_NAME` は `.env` ファイルに記載したモデルのデプロイ名で実行します。


In [ ]:
%pip install openai

In [ ]:
aoai_client = project_client.inference.get_azure_openai_client(
    api_version=os.getenv("MODEL_API_VERSION"),
)

response = aoai_client.chat.completions.create(
    model=os.getenv("MODEL_NAME"),  # Model deployment name
    messages=[
        {
            "role": "user",
            "content": "1マイルは何フィートですか?",
        },
    ],
)
print(response.choices[0].message.content)